In [ ]:
!pip3 install PyBullet

# Lab

The goal of this lab is to populate the pybullet server with real-time information from the Webots world and check whether there is a collision with projected motions of the robot in order to determine suitable grasps. 

You are provided with an URDF files of the crate and  the Robotiq gripper. You can use the gripper coordinate system to locate the crate and then perform relative motions with the gripper. You are also provided with a perfect object detection pipeline that segments all objects in the crate. 

Steps:

- Gain an understanding of how to load URDF and basic geometric objects into the PyBullet physics environment

- Understand the two ways that you can compute collisions: computing the closest points between two objects (does not require simulation step) and computing the actual contact points.

- Compute an Open3D point cloud, down sample it and approximate it with cubic or cyldrical objects. Also populate the box URDF into the same coordinate system. 

- Pick a single can, obtain its bounding object, and compute the position and orientation for grasping it. For this lab, you can limit yourself to grasping the cans at the center along its shorter width. 

- Test whether the grasp is feasible or not. If yes, execute it. If not, pick a new grasp. If no grasp is feasible, you can agitate the crate. 

Deliverable: A Python script that cycles through possible grasps and select a can it can grasps.

# PyBullet

PyBullet is the Python binding of the "Bullet" physics library. There exists multiple competing open-source physic simulation engines, another is the "Open Dynamics Engine" (ODE). This is the basis for the Webots simulator

This tutorial is focussed on getting you the necessary tools to use PyBullet to help your robot to reason about the physical world, such as detecting collisions or how physical objects might behave when the robot interact with it. Please refer to the PyBullet quickstart https://usermanual.wiki/Document/pybullet20quickstart20guide.479068914/html#pf28 for more information, specifically if you are interested how to simulate multi-body physics (arms), cameras, range finders etc. and to get an idea how Webots simulates these systems. 

## Example 1: Loading Objects from URDF

You can export URDF from Webots by right-clicking on an object in the scene tree. This example uses the Robotiq gripper and the fruit bowl that we are using in this lab. 

In [ ]:
import pybullet as p
import time
import pybullet_data

physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version

p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,-10) # set to (0,0,-10) for realistic gravity
planeId = p.loadURDF("plane.urdf")

startPosCrate = [0,0,0.02]
startOrientationCrate = p.getQuaternionFromEuler([0,0,0])
crateId = p.loadURDF("crate.urdf",startPosCrate, startOrientationCrate)

import numpy as np

for x in np.linspace(-0.15,0.15,20):
    startPos = [x,0,0.3]
    startOrientation = p.getQuaternionFromEuler([-1.57,0,0])
    gripperId = p.loadURDF("robotiq.urdf",startPos, startOrientation)
    p.resetBasePositionAndOrientation(crateId,startPosCrate,startOrientationCrate)    
    closestPoints=p.getClosestPoints(gripperId,crateId,0.01) # does not requrire simulation step
    if(closestPoints):
        print("Objects are in collision")
    else:
        print("Not in collision")
#    p.stepSimulation()    
    time.sleep(0.001)
    p.removeBody(gripperId)
    
    
#print(p.getMatrixFromQuaternion(startOrientation))
# It is also possible to use 'multiplyTransforms()' and 'invertTransforms()'

gripperId = p.loadURDF("robotiq.urdf",startPos, startOrientation)

for i in range (10000):
    p.stepSimulation()
    time.sleep(1./240.)
    p.resetBasePositionAndOrientation(crateId,startPosCrate,startOrientationCrate)
    if i%100 == 0:
        contacts=p.getContactPoints(gripperId,crateId)
        if contacts:
            print("Collision")

cubePos, cubeOrn = p.getBasePositionAndOrientation(bowlId)
print(cubePos,cubeOrn)
p.disconnect()            

## Example 2: Inserting standard geometries 

You can also place standard geometries in the environment. This can be useful to define simple collision objects. 

In [ ]:
import pybullet as p
import time
import pybullet_data

p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.createCollisionShape(p.GEOM_PLANE)
p.createMultiBody(0, 0)

sphereRadius = 0.05
colSphereId = p.createCollisionShape(p.GEOM_SPHERE, radius=sphereRadius)
colBoxId = p.createCollisionShape(p.GEOM_BOX,
                                  halfExtents=[sphereRadius, sphereRadius, sphereRadius])

mass = 1
visualShapeId = -1

link_Masses = [1]
linkCollisionShapeIndices = [colBoxId]
linkVisualShapeIndices = [-1]
linkPositions = [[0, 0, 0.11]]
linkOrientations = [[0, 0, 0, 1]]
linkInertialFramePositions = [[0, 0, 0]]
linkInertialFrameOrientations = [[0, 0, 0, 1]]
indices = [0]
jointTypes = [p.JOINT_REVOLUTE]
axis = [[0, 0, 1]]

for i in range(3):
  for j in range(3):
    for k in range(3):
      basePosition = [
          1 + i * 5 * sphereRadius, 1 + j * 5 * sphereRadius, 1 + k * 5 * sphereRadius + 1
      ]
      baseOrientation = [0, 0, 0, 1]
      if (k & 2):
        sphereUid = p.createMultiBody(mass, colSphereId, visualShapeId, basePosition,
                                      baseOrientation)
      else:
        sphereUid = p.createMultiBody(mass,
                                      colBoxId,
                                      visualShapeId,
                                      basePosition,
                                      baseOrientation,
                                      linkMasses=link_Masses,
                                      linkCollisionShapeIndices=linkCollisionShapeIndices,
                                      linkVisualShapeIndices=linkVisualShapeIndices,
                                      linkPositions=linkPositions,
                                      linkOrientations=linkOrientations,
                                      linkInertialFramePositions=linkInertialFramePositions,
                                      linkInertialFrameOrientations=linkInertialFrameOrientations,
                                      linkParentIndices=indices,
                                      linkJointTypes=jointTypes,
                                      linkJointAxis=axis)

      p.changeDynamics(sphereUid,
                       -1,
                       spinningFriction=0.001,
                       rollingFriction=0.001,
                       linearDamping=0.0)
      for joint in range(p.getNumJoints(sphereUid)):
        p.setJointMotorControl2(sphereUid, joint, p.VELOCITY_CONTROL, targetVelocity=1, force=10)

p.setGravity(0, 0, -10)
p.setRealTimeSimulation(1)

p.getNumJoints(sphereUid)
for i in range(p.getNumJoints(sphereUid)):
  p.getJointInfo(sphereUid, i)

while (1):
  keys = p.getKeyboardEvents()
  print(keys)

  time.sleep(0.01)

# Webots Recognition Node

Webots provides built-in object detection capability that works very similar to pixel-based segmentation tools such as YoLo, Mask R-CNN or Facebook's Detectron. Read up about it here https://www.cyberbotics.com/doc/reference/recognition

In this lab, we provide you with the ability to recognize the cans and pick out the one that you want to grasp. Specifically, the cans each have a distinct "recognition color". This emulates perfect segmentation such as can be obtained by DBScan or similar algorithms. All recognition colors are composed of 0, 0.5 and 1 (e.g., [0 0.5 0] or [1 1 0.5]).  


In [ ]:
from controller import Robot
import numpy as np

# create the Robot instance.
robot = Robot()

# get the time step of the current world.
timestep = int(robot.getBasicTimeStep())

speed=1.0

start_pos = [0,-1.382,-1.13, -2,1.63,3.142]

hand_motors=[robot.getDevice("finger_1_joint_1"),
             robot.getDevice("finger_2_joint_1"),
             robot.getDevice("finger_middle_joint_1")]

ur_motors=[robot.getDevice("shoulder_pan_joint"),
           robot.getDevice("shoulder_lift_joint"),
           robot.getDevice("elbow_joint"),
           robot.getDevice("wrist_1_joint"),
           robot.getDevice("wrist_2_joint"),
           robot.getDevice("wrist_3_joint")]

for i, ur_motor in enumerate(ur_motors):
    ur_motor.setVelocity(speed)
    ur_motor.setPosition(start_pos[i])

position_sensors=[robot.getDevice("shoulder_pan_joint_sensor"),
                  robot.getDevice("shoulder_lift_joint_sensor"),
                  robot.getDevice("elbow_joint_sensor"),
                  robot.getDevice("wrist_1_joint_sensor"),
                  robot.getDevice("wrist_2_joint_sensor"),
                  robot.getDevice("wrist_3_joint_sensor")]

for position_sensor in position_sensors:
    position_sensor.enable(timestep)
    
camera = robot.getDevice("camera")
camera.enable(timestep)
camera.recognitionEnable(timestep)
camera.enableRecognitionSegmentation()

rangefinder = robot.getDevice("range-finder")
rangefinder.enable(timestep)                     


for i in range(200):
    robot.step(timestep)


## Read depth and segmented image

In [ ]:
depth_1darray = np.frombuffer(rangefinder.getRangeImage(data_type="buffer"), dtype=np.float32)
depth=np.reshape(depth_1darray,(240,320))
depth=depth*1000.0

image_1darray = camera.getRecognitionSegmentationImage()
image = np.frombuffer(image_1darray, np.uint8).reshape((camera.getHeight(), camera.getWidth(), 4))

mono = np.dot(image,[0.2989, 0.5870, 0.1140,0]).astype(int)

robot.step(timestep)

## Mask depth image with segmented image

The goal of segmentation is to make individual objects uniquely identifiable. In this case, the segmentation routine will label each object's pixel with a unique color. We can use this information to mask these objects in the depth image.

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(1, 3, 1)
plt.title('Camera image')
plt.imshow(image)
plt.subplot(1, 3, 2)
plt.title('Depth image')
plt.imshow(depth)
plt.subplot(1, 3, 3)
plt.title('Monochrome image')
plt.imshow(mono)
plt.show()

In [ ]:
unique_colors=np.unique(mono).tolist()
unique_colors.pop(0) # remove the first color - zero

fig, ax = plt.subplots(len(unique_colors),1,figsize=(24,32))
fig.tight_layout()

for i, color in enumerate(unique_colors):
    cand=np.multiply(depth,mono == color)
    ax[i].imshow(cand)
#    plt.title('Color {}'.format(color))
plt.show()

# Compute Object Pose from Open3D Pointcloud

We can pick individual objects by multiplying the depth image with a mask. We can hence compute the oriented bounding box and the relative pose of the object for the robot to grasp. 

The oriented bounding box data structure provides the center (.center) and orientation (.R), which allows you to compute its pose in the robot's base frame. You can then compute an IK solution that moves the robot arm there. 

In [ ]:
cand=np.multiply(depth,mono == 254) # pick the white can

In [ ]:
import open3d as o3d

rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        o3d.geometry.Image(image), 
        o3d.geometry.Image(np.array(cand).astype('uint16')),
        convert_rgb_to_intensity=False,
        depth_scale=1000.0, depth_trunc=1.5)


can = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
       o3d.camera.PinholeCameraIntrinsic(320,240,320,240,160,120),
    project_valid_depth_only=True
)

can.paint_uniform_color([1.0, 0, 0]) # you need to color this as the segmentation color is unreliable, in particular for the white object

#o3d.visualization.draw_geometries([can])

obb = can.get_oriented_bounding_box()
obb.color = (0, 1, 0)
#o3d.visualization.draw_geometries([obb,can])


In [ ]:
# building the transformation matrix from rotation, translation, and [0 0 0 1]

Randt=np.concatenate((obb.R, np.expand_dims(obb.center, axis=1)),axis=1) # pitfall: arrays need to be passed as a tuple
lastrow=np.expand_dims(np.array([0,0,0,1]),axis=0)
T=np.concatenate((Randt,lastrow))
print(T)

# Compute grasp pose using Forward Kinematics

So far, we computed the position of the can only in end-effector space. The robot's inverse kinematics expects coordinates in the robot's frame. Note that the robot's frame that the IK solver is using is not visible in Webots. Its origin is around 7cm above the origin of the UR5e's base in Webots. 

Obviously, we cannot directly move the end-effector to the can. We will need to come a little from above, taking into account the lengt of the gripper. By experimenting with the system, we find out that we need to rotate the can by 90 degrees in order to grasp it along the narrow dimension. 

You will need to apply one more rotation to have the gripper point down, not up.


In [ ]:
T[2,3]-=0.30 # move 20cm above the can

q=-np.pi/2
Rz=[[np.cos(q),-np.sin(q),0,0],
    [np.sin(q),np.cos(q),0,0],
    [0,0,1,0],
    [0,0,0,1]]
print(T)
T=np.matmul(T,Rz)

#
#
#  Add another rotation here
#
#

print(T)


In [ ]:
import cmath
import math
from math import cos as cos
from math import sin as sin
from math import atan2 as atan2
from math import acos as acos
from math import asin as asin
from math import sqrt as sqrt
from math import pi as pi

global mat
mat=np.matrix

global d, a, alph

d = mat([0.1625, 0, 0, 0.1333, 0.0997, 0.0996]) #ur5e
a = mat([0 ,-0.425 ,-0.3922 ,0 ,0 ,0]) #ur5e
alph = mat([math.pi/2, 0, 0, math.pi/2, -math.pi/2, 0 ])  #ur5e

# ************************************************** FORWARD KINEMATICS

def AH( n,th,c  ):

  T_a = mat(np.identity(4), copy=False)
  T_a[0,3] = a[0,n-1]
  T_d = mat(np.identity(4), copy=False)
  T_d[2,3] = d[0,n-1]

  Rzt = mat([[cos(th[n-1,c]), -sin(th[n-1,c]), 0 ,0],
	         [sin(th[n-1,c]),  cos(th[n-1,c]), 0, 0],
	         [0,               0,              1, 0],
	         [0,               0,              0, 1]],copy=False)
      

  Rxa = mat([[1, 0,                 0,                  0],
			 [0, cos(alph[0,n-1]), -sin(alph[0,n-1]),   0],
			 [0, sin(alph[0,n-1]),  cos(alph[0,n-1]),   0],
			 [0, 0,                 0,                  1]],copy=False)

  A_i = T_d * Rzt * T_a * Rxa
	    

  return A_i

def HTrans(th,c ):  
  A_1=AH( 1,th,c  )
  A_2=AH( 2,th,c  )
  A_3=AH( 3,th,c  )
  A_4=AH( 4,th,c  )
  A_5=AH( 5,th,c  )
  A_6=AH( 6,th,c  )
      
  T_06=A_1*A_2*A_3*A_4*A_5*A_6
  return T_06

def get_joint_angles():
    th=[]
    for position_sensor in position_sensors:
        th.append(position_sensor.getValue())
    return th

def restorePose(angles):
    for i,ur_motor in enumerate(ur_motors):
        ur_motor.setPosition(angles[i])

    for i in range(200):
        robot.step(timestep)

In [ ]:
# 1. Obtain current joint angles
# 2. compute end-effector pose
# 3. transform desired pose into robot frame
robot.step(timestep)
angles=get_joint_angles()
theta=np.matrix([[theta] for theta in angles])
c=[0]
InitialPose=HTrans(theta,c)

DesiredPose=np.matmul(HTrans(theta,c),T) # Transform can pose T into robot coordinate frame

In [ ]:
print(np.round(InitialPose,decimals=2))

In [ ]:
print(np.round(DesiredPose,decimals=2))

# Implement motion

We can now use IK to compute the joint angles that we need to get to. 

In [ ]:
# ************************************************** INVERSE KINEMATICS 

from numpy import linalg

global d1, a2, a3, a7, d4, d5, d6
d1 =  0.1625
a2 = -0.425
a3 = -0.3922
d4 =  0.1333
d5 =  0.0997
d6 =  0.0996

def invKine(desired_pos):# T60
  th = mat(np.zeros((6, 8)))
  P_05 = (desired_pos * mat([0,0, -d6, 1]).T-mat([0,0,0,1 ]).T)
  
  # **** theta1 ****
  
  psi = atan2(P_05[2-1,0], P_05[1-1,0])
  phi = acos(d4 /sqrt(P_05[2-1,0]*P_05[2-1,0] + P_05[1-1,0]*P_05[1-1,0]))
  #The two solutions for theta1 correspond to the shoulder
  #being either left or right
  th[0, 0:4] = pi/2 + psi + phi
  th[0, 4:8] = pi/2 + psi - phi
  th = th.real
  
  # **** theta5 ****
  
  cl = [0, 4]# wrist up or down
  for i in range(0,len(cl)):
	      c = cl[i]
	      T_10 = linalg.inv(AH(1,th,c))
	      T_16 = T_10 * desired_pos
	      th[4, c:c+2] = + acos((T_16[2,3]-d4)/d6);
	      th[4, c+2:c+4] = - acos((T_16[2,3]-d4)/d6);

  th = th.real
  
  # **** theta6 ****
  # theta6 is not well-defined when sin(theta5) = 0 or when T16(1,3), T16(2,3) = 0.

  cl = [0, 2, 4, 6]
  for i in range(0,len(cl)):
	      c = cl[i]
	      T_10 = linalg.inv(AH(1,th,c))
	      T_16 = linalg.inv( T_10 * desired_pos )
	      th[5, c:c+2] = atan2((-T_16[1,2]/sin(th[4, c])),(T_16[0,2]/sin(th[4, c])))
		  
  th = th.real

  # **** theta3 ****
  cl = [0, 2, 4, 6]
  for i in range(0,len(cl)):
	      c = cl[i]
	      T_10 = linalg.inv(AH(1,th,c))
	      T_65 = AH( 6,th,c)
	      T_54 = AH( 5,th,c)
	      T_14 = ( T_10 * desired_pos) * linalg.inv(T_54 * T_65)
	      P_13 = T_14 * mat([0, -d4, 0, 1]).T - mat([0,0,0,1]).T
	      t3 = cmath.acos((linalg.norm(P_13)**2 - a2**2 - a3**2 )/(2 * a2 * a3)) # norm ?
	      th[2, c] = t3.real
	      th[2, c+1] = -t3.real

  # **** theta2 and theta 4 ****

  cl = [0, 1, 2, 3, 4, 5, 6, 7]
  for i in range(0,len(cl)):
	      c = cl[i]
	      T_10 = linalg.inv(AH( 1,th,c ))
	      T_65 = linalg.inv(AH( 6,th,c))
	      T_54 = linalg.inv(AH( 5,th,c))
	      T_14 = (T_10 * desired_pos) * T_65 * T_54
	      P_13 = T_14 * mat([0, -d4, 0, 1]).T - mat([0,0,0,1]).T
	      
	      # theta 2
	      th[1, c] = -atan2(P_13[1], -P_13[0]) + asin(a3* sin(th[2,c])/linalg.norm(P_13))
	      # theta 4
	      T_32 = linalg.inv(AH( 3,th,c))
	      T_21 = linalg.inv(AH( 2,th,c))
	      T_34 = T_32 * T_21 * T_14
	      th[3, c] = atan2(T_34[1,0], T_34[0,0])
  th = th.real

  return th

In [ ]:
# select the solution with the lowest joint motion, weighted strongest by the shoulder lift joint
score=[]
iksolution=invKine(DesiredPose)
for i in range(max(iksolution.shape)):
    goal=iksolution[:,i]
    goal=[angle.item() for angle in goal]
    score.append(sum(np.multiply([3,5,4,3,2,1],np.sqrt(np.square(np.array(angles)-np.array(goal))))))
    print("Solution: {} Score {}".format(i,score[i]))
    #restorePose(goal)
          
    

In [ ]:
best_goal = iksolution[:,score.index(min(score))]
best_goal=[angle.item() for angle in best_goal]

In [ ]:
#restorePose(best_goal) # a cheap way to achieve moveJ
restorePose(angles)